In [ ]:
# IMPORTAÇÕES
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string
import unicodedata



In [ ]:
# Carregando dados
vagas = pd.read_json('vagas.json').T
applicants = pd.read_json('applicants.json').T
prospects = pd.read_json('prospects.json').T

In [ ]:
vagas.reset_index(inplace=True)
applicants.reset_index(inplace=True)
prospects.reset_index(inplace=True)

In [ ]:
vagas = vagas.rename(columns={'index': 'vaga_id'})
applicants = applicants.rename(columns={'index': 'codigo_candidato'})
prospects = prospects.rename(columns={'index': 'codigo_vaga'})


In [ ]:
# NORMALIZAR VAGAS
o_infobasica = pd.json_normalize(vagas['informacoes_basicas'])
o_perfil = pd.json_normalize(vagas['perfil_vaga'])
o_beneficios = pd.json_normalize(vagas['beneficios'])

vagas_df = pd.concat([vagas, o_infobasica, o_perfil, o_beneficios], axis=1)
vagas_df = vagas_df.drop(columns=['informacoes_basicas', 'perfil_vaga', 'beneficios', 'url_vaga', 'nome_cliente', 'modalidade', 'titulo'], errors='ignore')

vagas_df.head()

,vaga_id,data_requicisao,limite_esperado_para_contratacao,titulo_vaga,vaga_sap,cliente,solicitante_cliente,empresa_divisao,requisitante,analista_responsavel,...,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,viagens_requeridas,equipamentos_necessarios,habilidades_comportamentais_necessarias,valor_venda,valor_compra_1,valor_compra_2
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,NaN,-,R$,
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,...,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,,Nenhum -,NaN,-,R$,
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,...,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,NaN,-,R$,
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,...,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,NaN,- p/ mês (168h),fechado,
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,NaN,-,R$,


In [ ]:
# NORMALIZAR APPLICANTS
infos_basicas = pd.json_normalize(applicants['infos_basicas'])
informacoes_pessoais = pd.json_normalize(applicants['informacoes_pessoais'])
informacoes_profissionais = pd.json_normalize(applicants['informacoes_profissionais'])
formacao_e_idiomas = pd.json_normalize(applicants['formacao_e_idiomas'])
cargo_atual = pd.json_normalize(applicants['cargo_atual'])

applicants_df = pd.concat([applicants, infos_basicas, informacoes_pessoais, informacoes_profissionais, formacao_e_idiomas, cargo_atual], axis=1)
applicants_df = applicants_df.drop(columns=['infos_basicas', 'informacoes_pessoais', 'informacoes_profissionais', 'formacao_e_idiomas', 'cargo_atual', 'email', 'telefone', 'foto', 'carta_apresentacao'], errors='ignore')

applicants_df.head()

,codigo_candidato,cv_pt,cv_en,telefone_recado,objetivo_profissional,data_criacao,inserido_por,local,sabendo_de_nos_por,data_atualizacao,...,outro_curso,id_ibrati,email_corporativo,projeto_atual,cliente,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,email_superior_imediato
0,31000,assistente administrativo\n\n\nsantosbatista\n...,,,,10-11-2021 07:29:49,Luna Correia,,,10-11-2021 07:29:49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31001,formação acadêmica\nensino médio (2º grau) em ...,,,Analista Administrativo,10-11-2021 08:56:16,Laura Pacheco,"São Paulo, São Paulo",Outros,11-11-2021 11:10:31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31002,objetivo: área administrativa | financeira\n\n...,,,Administrativo | Financeiro,10-11-2021 09:01:00,Laura Pacheco,"São Paulo, São Paulo",Anúncio,10-11-2021 11:42:36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31003,formação\nensino médio completo\ninformática i...,,,Área administrativa,10-11-2021 09:08:13,Laura Pacheco,"São Paulo, São Paulo",Site de Empregos,10-11-2021 16:04:51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31004,última atualização em 09/11/2021\n­ sp\n\nensi...,,,,10-11-2021 09:18:46,Maria Clara Pires,,,10-11-2021 09:22:03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
applicants_df.columns

Index(['codigo_candidato', 'cv_pt', 'cv_en', 'telefone_recado',
       'objetivo_profissional', 'data_criacao', 'inserido_por', 'local',
       'sabendo_de_nos_por', 'data_atualizacao', 'codigo_profissional', 'nome',
       'data_aceite', 'nome', 'cpf', 'fonte_indicacao', 'email_secundario',
       'data_nascimento', 'telefone_celular', 'telefone_recado', 'sexo',
       'estado_civil', 'pcd', 'endereco', 'skype', 'url_linkedin', 'facebook',
       'download_cv', 'titulo_profissional', 'area_atuacao',
       'conhecimentos_tecnicos', 'certificacoes', 'outras_certificacoes',
       'remuneracao', 'nivel_profissional', 'qualificacoes', 'experiencias',
       'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma',
       'instituicao_ensino_superior', 'cursos', 'ano_conclusao', 'outro_curso',
       'id_ibrati', 'email_corporativo', 'projeto_atual', 'cliente', 'unidade',
       'data_admissao', 'data_ultima_promocao', 'nome_superior_imediato',
       'email_superior_imediato']

In [ ]:
# Verificar percentual de valores nulos por coluna
missing_percent = applicants_df.isnull().mean().sort_values(ascending=False) * 100

# Mostrar apenas colunas que tem valores faltantes
print(missing_percent[missing_percent > 0])

outro_curso                    98.095664
data_ultima_promocao           97.777882
experiencias                   97.777882
cliente                        97.777882
qualificacoes                  97.777882
data_admissao                  97.777882
unidade                        97.777882
email_superior_imediato        97.777882
projeto_atual                  97.777882
email_corporativo              97.777882
nome_superior_imediato         97.777882
id_ibrati                      97.777882
download_cv                    86.907396
ano_conclusao                  83.030460
cursos                         83.030460
instituicao_ensino_superior    83.030460
dtype: float64


In [ ]:
#NORMALIZAR PROSPECTS
prospects_exploded = prospects.explode('prospects')
prospects_normalized = pd.json_normalize(prospects_exploded['prospects'])
prospects_transform = pd.concat([
    prospects_exploded.reset_index(drop=True).drop(columns=['prospects']),
    prospects_normalized.reset_index(drop=True)
], axis=1)

prospects_transform.head()

,codigo_vaga,titulo,modalidade,nome,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador
0,4530,CONSULTOR CONTROL M,,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira
1,4530,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano
4,4532,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
prospects_transform.columns

Index(['codigo_vaga', 'titulo', 'modalidade', 'nome', 'codigo',
       'situacao_candidado', 'data_candidatura', 'ultima_atualizacao',
       'comentario', 'recrutador'],
      dtype='object')

In [ ]:
vagas_df.head()

,vaga_id,data_requicisao,limite_esperado_para_contratacao,titulo_vaga,vaga_sap,cliente,solicitante_cliente,empresa_divisao,requisitante,analista_responsavel,...,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,viagens_requeridas,equipamentos_necessarios,habilidades_comportamentais_necessarias,valor_venda,valor_compra_1,valor_compra_2
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,NaN,-,R$,
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,...,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,,Nenhum -,NaN,-,R$,
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,...,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,NaN,-,R$,
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,...,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,NaN,- p/ mês (168h),fechado,
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,NaN,-,R$,


In [ ]:
vagas_df.columns

Index(['vaga_id', 'data_requicisao', 'limite_esperado_para_contratacao',
       'titulo_vaga', 'vaga_sap', 'cliente', 'solicitante_cliente',
       'empresa_divisao', 'requisitante', 'analista_responsavel',
       'tipo_contratacao', 'prazo_contratacao', 'objetivo_vaga',
       'prioridade_vaga', 'origem_vaga', 'superior_imediato', 'nome',
       'telefone', 'data_inicial', 'data_final', 'nome_substituto', 'pais',
       'estado', 'cidade', 'bairro', 'regiao', 'local_trabalho',
       'vaga_especifica_para_pcd', 'faixa_etaria', 'horario_trabalho',
       'nivel profissional', 'nivel_academico', 'nivel_ingles',
       'nivel_espanhol', 'outro_idioma', 'areas_atuacao',
       'principais_atividades', 'competencia_tecnicas_e_comportamentais',
       'demais_observacoes', 'viagens_requeridas', 'equipamentos_necessarios',
       'habilidades_comportamentais_necessarias', 'valor_venda',
       'valor_compra_1', 'valor_compra_2'],
      dtype='object')

In [ ]:
contratados = vagas_prospects[vagas_prospects['situacao_candidado'].str.contains('Contratado', case=False, na=False)].copy()
contratados = contratados.rename(columns={'codigo_vaga': 'vaga_id', 'codigo_id': 'codigo'})

In [ ]:
contratados['codigo'] = contratados['codigo'].astype(str)
applicants_df['codigo_candidato'] = applicants_df['codigo_candidato'].astype(str)

In [ ]:
textos = dataset['cv_pt_tratado'].fillna('')

In [ ]:
from google.colab import files
files.download('dataset_completo.csv')

In [ ]:
vagas_df.to_csv("vagas.csv", index= False)

In [ ]:
from google.colab import files
files.download('vagas.csv')

In [ ]:
applicants_df.to_csv("applicants.csv", index= False)

In [ ]:
from google.colab import files
files.download('applicants.csv')

In [ ]:
prospects_transform.to_csv("prospects.csv", index= False)

In [ ]:
from google.colab import files
files.download('prospects.csv')